In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plot
from image_utils import label_images, plot_images, count_samples

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import Model, Sequential, models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

np.random.seed(6)

In [ ]:
# # verify if tensorflow is running with GPU
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

# # check if keras is using GPU
print(K.tensorflow_backend._get_available_gpus())

## Define data, sample size, and image parameters

In [ ]:
# define datasets
train_data_dir = "/kaggle/input/intel-image-classification/seg_train/seg_train/"
test_data_dir = "/kaggle/input/intel-image-classification/seg_test/seg_test/"
pred_data_dir = "/kaggle/input/intel-image-classification/seg_pred/seg_pred/"
 
# define datasets and sample size
num_train_samples = count_samples(train_data_dir)
num_test_samples = count_samples(test_data_dir)
num_pred_samples = count_samples(pred_data_dir, pred = "yes")

# define image dimensions
img_width, img_height = 150, 150
img_target_size = (img_width, img_height)

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

## Plot images

In [ ]:
# # label images
# training_imgs, labeled_training_imgs = label_images(train_data_dir, '.jpg')
# plot_images(training_imgs, labeled_training_imgs)

## Define hyperparameters and data generators

In [ ]:
# define model hyperparameters
epochs = 50
batch_size = 96
patience = 3

# specify image augmentation
img_augmentation = ImageDataGenerator(rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True)

# prepare train/test data generators
train_data = img_augmentation.flow_from_directory(
    train_data_dir,
    target_size = img_target_size,
    batch_size = batch_size,
    class_mode = 'categorical')

test_data = img_augmentation.flow_from_directory(
    test_data_dir,
    target_size = img_target_size,
    batch_size = batch_size,
    class_mode = 'categorical')

pred_data = img_augmentation.flow_from_directory(
    pred_data_dir,
    target_size = img_target_size,
    batch_size = batch_size,
    class_mode = None,
    shuffle = False)

## Create and train model

In [ ]:
model = Sequential()
model.add(layers.Conv2D(128, 3, input_shape=input_shape, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Dropout(.10))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))
model.add(layers.Dropout(.10))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(128, 3, activation="relu"))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))
model.add(layers.Dropout(.10))
model.add(layers.Conv2D(96, 3, activation="relu"))
model.add(layers.Conv2D(64, 3, activation="relu"))
model.add(layers.Conv2D(64, 3, activation="relu"))
model.add(layers.Conv2D(48, 3, activation="relu"))
model.add(layers.Conv2D(48, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Dropout(.10))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.20))
model.add(layers.Dense(6, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['acc'])

callbacks = [EarlyStopping(monitor='val_acc', patience=patience),
             ModelCheckpoint(filepath='cnnc_iic_kaggle.h5', monitor='acc', save_best_only=True)]

# train model
history = model.fit_generator(
    train_data,
    steps_per_epoch = num_train_samples // batch_size,
#     validation_data = test_data,
#     validation_steps = num_test_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    verbose = 1,
    use_multiprocessing = True)

## Plot training vs val accuracy

In [ ]:
plot.plot(history.history['acc'])
plot.plot(history.history['val_acc'])
plot.title('Model accuracy')
plot.ylabel('Accuracy')
plot.xlabel('Epoch')
plot.legend(['Train', 'Test'], loc='upper left')
plot.show()

## Perform CV

In [ ]:
test_score = model.evaluate_generator(test_data,
    steps = num_test_samples // batch_size,
    verbose=1,
    use_multiprocessing=True)

print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

## Make predictions

In [ ]:
pred_score = model.predict_generator(pred_data,
    steps = num_pred_samples // batch_size,
    verbose=1,
    use_multiprocessing=True)

print(pred_score)